In [1]:
### LOAD ###
import json
import pandas as pd
import en_core_web_sm
import spacy

debates = []

for line in open('../data/debate.jsonl', 'r'):
    debates.append(json.loads(line))

debates_ = pd.DataFrame(debates)
debates_

,id,titles,arguments,counters
0,t3_2ro9ux,Anything that is manmade is natural.,I cant remember the topic that spurred this di...,Look at the definition you provided if we remo...
1,t3_2ro0ti,Essential Oils are bullshit,y wife has recently gotten deeply involvedobse...,This answer in raskscience does a pretty good ...
2,t3_2rnr30,I think the Paris shooting makes a good case f...,Thinking about todays news BB Wikipedia I thin...,I guess I look at it this way there have absol...
3,t3_2rnfn0,Explaining causation is not blaming the victim...,Ive been thinking about this issue for a while...,I would say that the difficulty here is in ass...
4,t3_2rnfn0,Explaining causation is not blaming the victim...,Ive been thinking about this issue for a while...,I think youre correct for the most part but st...
...,...,...,...,...
3451,t3_2rsgv3,Drawing images of ohammed and posting them on ...,In opposing injustice we must strive not to pe...,Its an issue of context. If the purpose is to ...
3452,t3_2rs57a,The rate at which ones income is taxed should ...,The rate at which income is taxed at least in ...,Their takehome pay will still be directly tied...
3453,t3_2rs57a,The rate at which ones income is taxed should ...,The rate at which income is taxed at least in ...,This is a common misconception. The system is ...
3454,t3_2rqvf8,School hours should be am to pm to match offic...,It seems logical to me that school hours shoul...,They already have these. Every school in my lo...


In [9]:
# <OP, Counter Argument> Pairs, for Seq2Seq Modeling
arguments = list(debates_["arguments"])
counters = list(debates_["counters"])

arguments

['I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural. He took the position that they are unnatural. He cited this definition by erriamWebster existing in nature and not made or caused by people coming from nature  as his basis for the distinction for natural vs. unnatural. However I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankinds capacity to create problemsolve analyze rationalize and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability why is it that we would give the credit of these things solely to man as opposed to nature? We are not separate from nature thus how can any of our actions or creations be unnatural? If we were somehow separate from nature I would understand the distinction between natural and manmade. However I think unnatural and manm

In [3]:
# Tokenise
nlp = spacy.load('en_core_web_sm')

def tokenise(text):
    tokens = []

    for i in text:
        doc_ = nlp(i)
        toks = [token.text for token in doc_]
        bos_token = '<BOS>'
        eos_token = '<EOS>'

        toks.insert(0, bos_token)
        toks.append(eos_token)
        tokens.append(toks)

    return tokens

In [82]:
arguments_ = tokenise(arguments)

In [4]:
counters_ = tokenise(counters)

In [86]:
len(arguments_), len(counters_)
assert len(arguments_) == len(counters_), "Lengths of encoder source data and decoder target data are not equal!"

In [7]:
### FILE OUT ###
import re

# Provide only output sentences for GPT-2
train = int(len(counters_)*0.8)
valid = len(counters_) - train

def file_out(output_path, data):

    with open(output_path, "w") as fout:
        for i in data:
            fout.write(str(i).strip("[]"))
            fout.write("\n")

train = file_out("../data/gpt/train_tok.txt", counters_[0:train])
test = file_out("../data/gpt/valid_tok.txt", counters_[-valid:])

In [11]:
### NON-TOKENISED ###

train = int(len(counters)*0.8)
valid = len(counters) - train

def file_out(output_path, data):

    with open(output_path, "w") as fout:
        for i in data:
            fout.write(str(i).strip("[]"))
            fout.write("\n")

train = file_out("../data/gpt/train.txt", counters[0:train])
test = file_out("../data/gpt/valid.txt", counters[-valid:])